# Extract Poses into NPY Files (Mimic PoseRAC's pre_test.py)

In [1]:
import numpy as np
import pandas as pd
import glob
import re
import os
import sys
import deeplabcut

2024-09-07 16:09:25.098072: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-07 16:09:26.189204: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:
2024-09-07 16:09:26.189286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:
2024-09-07 16:09:26.189293: W tensorflow/compiler/tf2tensorrt/utils/py_uti

Loading DLC 2.3.10...


In [9]:
# System alterations

# Set the print options to avoid truncation
np.set_printoptions(threshold=sys.maxsize) # Default threshold is 1000

In [2]:
# Note that parameters of this project can be seen at: *openfield-Pranav-2018-10-30/config.yaml*
PATH_CONFIG_FILE = os.path.join(os.getcwd(), 'examples', 'CowBytes-Single-Sadat-2024-06-30', 'config.yaml')
deeplabcut.load_demo_data(PATH_CONFIG_FILE)

This is not an official demo dataset.
Loaded, now creating training data...
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [3]:
# Define test poses destination directory and video file paths [EDIT HERE]
TEST_POSES_DEST_DIR = os.path.join(os.getcwd(), 'examples', 'BiteCountA_pose', 'test_poses')
ROOT_VIDEO_DIR = os.path.join(os.getcwd(), 'examples', 'BiteCountA_pose', 'video')
TRAIN_VIDEO_DIR = os.path.join(ROOT_VIDEO_DIR, 'train')
VALID_VIDEO_DIR = os.path.join(ROOT_VIDEO_DIR, 'valid')
TEST_VIDEO_DIR = os.path.join(ROOT_VIDEO_DIR, 'test')

In [5]:
# Create test_poses directory
os.makedirs(TEST_POSES_DEST_DIR, exist_ok=True)

In [ ]:
# Extract poses into CSV files
video_dir = [TRAIN_VIDEO_DIR, VALID_VIDEO_DIR, TEST_VIDEO_DIR]

for dir in video_dir:
    deeplabcut.analyze_videos(PATH_CONFIG_FILE,dir,save_as_csv=True)

In [4]:
def normalise(datapoints, dimensions):
    x_max = np.expand_dims(np.max(datapoints[:, 0::3], axis=1), 1)
    x_min = np.expand_dims(np.min(datapoints[:, 0::3], axis=1), 1)

    y_max = np.expand_dims(np.max(datapoints[:, 1::3], axis=1), 1)
    y_min = np.expand_dims(np.min(datapoints[:, 1::3], axis=1), 1)

    datapoints[:, 0::3] = (datapoints[:, 0::3] - x_min) / (x_max - x_min)
    datapoints[:, 1::3] = (datapoints[:, 1::3] - y_min) / (y_max - y_min)

    if dimensions == 3:
        z_max = np.expand_dims(np.max(datapoints[:, 2::3], axis=1), 1)
        z_min = np.expand_dims(np.min(datapoints[:, 2::3], axis=1), 1)
        datapoints[:, 2::3] = (datapoints[:, 2::3] - z_min) / (z_max - z_min)
    else:
        datapoints[:, 2::3] = 0
    return datapoints

In [2]:
# Extract depth estimations in MiDaS environment and move .pfm files to extracted directory

In [5]:
# https://github.com/isl-org/MiDaS/blob/master/utils.py
def read_pfm(path):
    """Read pfm file.

    Args:
        path (str): path to file

    Returns:
        tuple: (data, scale)
    """
    with open(path, "rb") as file:

        color = None
        width = None
        height = None
        scale = None
        endian = None

        header = file.readline().rstrip()
        if header.decode("ascii") == "PF":
            color = True
        elif header.decode("ascii") == "Pf":
            color = False
        else:
            raise Exception("Not a PFM file: " + path)

        dim_match = re.match(r"^(\d+)\s(\d+)\s$", file.readline().decode("ascii"))
        if dim_match:
            width, height = list(map(int, dim_match.groups()))
        else:
            raise Exception("Malformed PFM header.")

        scale = float(file.readline().decode("ascii").rstrip())
        if scale < 0:
            # little-endian
            endian = "<"
            scale = -scale
        else:
            # big-endian
            endian = ">"

        data = np.fromfile(file, endian + "f")
        shape = (height, width, 3) if color else (height, width)

        data = np.reshape(data, shape)
        data = np.flipud(data)

        return data, scale

In [6]:
# Load all generated .csv files
csv_files = glob.glob(os.path.join(ROOT_VIDEO_DIR, '**', '*.csv'), recursive=True)
print("Number of generated CSV Files:", len(csv_files))
print(csv_files[:5])

Number of generated CSV Files: 14
['/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/test/4ff8a74c52c529eceb817a1d89e76a3ae053583c2abc491ecacbe560484b2e7e_2DLC_resnet50_CowBytes-SingleJun30shuffle1_288000.csv', '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/test/d40739c2aa4b801a2eda2bbb8a554a6ff85e3fc331b719fb52af72bf43224086_2DLC_resnet50_CowBytes-SingleJun30shuffle1_288000.csv', '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/test/747419b046c70c69e41b2857b7af91465dcc0c49e0e91ca96f253da03067dd32_2DLC_resnet50_CowBytes-SingleJun30shuffle1_288000.csv', '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/test/10f1a9b90589c2bc657320dc4958fb19ea2d51fbcdb251402e6112e4407de2e7_4DLC_resnet50_CowBytes-SingleJun30shuffle1_288000.csv', '/home/jason/Documents/repositories/DeepLabCut/examples/BiteCountA_pose/video/test/e151c20b878c4425ce832d2f5120a218f4e31b765ec2a352130a5a877fddd041_2

In [ ]:
for csv_file in csv_files:
    # Process CSV file contents
    df = pd.read_csv(csv_file, header=None)
    df = df.drop([0, 1, 2]).reset_index(drop=True)
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(np.float32)
    df.iloc[:, 2::3] = 0 # Set z-values to 0

    # Get video name from CSV file name
    video_name = os.path.basename(csv_file.split('DLC')[0])

    # Get depth values from PFM files
    for i, row in df.iterrows():
        pfm_file = os.path.join(os.path.dirname(csv_file), f"{video_name}_{str(i)}-dpt_swin2_large_384.pfm")
        if os.path.isfile(pfm_file):
            data, _ = read_pfm(pfm_file)
            x_coords = np.rint(row[0::3].to_numpy().astype(np.float32)).astype(int)
            y_coords = np.rint(row[1::3].to_numpy().astype(np.float32)).astype(int)
            x_coords = np.clip(x_coords, 0, data.shape[1] - 1)
            y_coords = np.clip(y_coords, 0, data.shape[0] - 1)
            z_values = data[y_coords, x_coords]
            row[2::3] = z_values

    datapoints = normalise(df.values.astype(np.float32), 3)

    # Save NPY file with datapoints
    dest_file = os.path.join(TEST_POSES_DEST_DIR, video_name)
    np.save(dest_file, datapoints)
    print("Created file: ", dest_file)

In [ ]:
# Remove redundant files in video directories
files = glob.glob(os.path.join(ROOT_VIDEO_DIR, '**', '*.*'), recursive=True)

# Delete non-MP4 files
for file in files:
    if not file.endswith('.mp4'):
        try:
            os.remove(file)
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")